In [ ]:
from ROOT import *
import numpy as np
import math
import time
import matplotlib.pyplot as plt

from shapely.geometry import LineString
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

gROOT.ProcessLine(".x ~/lhcbStyle.C")
gStyle.SetPaintTextFormat("1.3f")

ch1=TChain("MCParticleNTuple/Tracks")
ch1.Add("~/MightyIT/MCtracks_MagUp_bs2phiphi_1p5e34_nocuts_20ev.root")
ch2=TChain("MCParticleNTuple/Tracks")
ch2.Add("~/MightyIT/MCtracks_MagUp_bs2phiphi_1p5e34_nocuts_20ev.root")

#FILES WITH CUTS IN ETA AND CUTS IN SECONDARIES
#ch1=TChain("MCParticleNTuple/Tracks")
#ch1.Add("~/MightyIT/MCtracks_MagUp_bs2phiphi_1p5e34_20ev.root")
#ch2=TChain("MCParticleNTuple/Tracks")
#ch2.Add("~/MightyIT/MCtracks_MagUp_bs2phiphi_1p5e34_20ev.root")



In [ ]:
def removezeros(listwithzerosX,listwithzerosY,listwithzerosZ):
    listwithoutzerosX = []
    listwithoutzerosY = []
    listwithoutzerosZ = []
    for item in range(0,len(listwithzerosZ)):
        if listwithzerosZ[item] > 0:
            listwithoutzerosX.append(listwithzerosX[item])
            listwithoutzerosY.append(listwithzerosY[item])
            listwithoutzerosZ.append(listwithzerosZ[item])
    return listwithoutzerosX,listwithoutzerosY,listwithoutzerosZ

In [ ]:
def whichconfig(uzlist,vzlist):
    isassigned = False
    if len(uzlist)>0 and len(vzlist)>0:
        uz0 = uzlist[0]; vz0 = vzlist[len(vzlist)-1]
        if (uz0-vz0)!=0 and 0<vz0<800 and 2000<uz0<3000:
            isassigned=True
            return 'VeloUT' #best because most precise slope (long)
    if len(uzlist)>1:
        uz0 = uzlist[0]; uz1 = uzlist[len(uzlist)-1]
        if (uz1-uz0)!= 0 and 2000<uz0<uz1<3000:
            isassigned=True
            return 'UTUT' #better than velovelo because hits are closer -> error propagated less
    if len(vzlist)>1:
        vz0 = vzlist[0]; vz1 = vzlist[len(vzlist)-1]
        if (vz1-vz0)!=0 and 0<vz0<vz1<800:
            isassigned=True
            return 'VeloVelo'
    if isassigned == False:
        return -1
    

def isvelout(uzlist,vzlist):
    isassigned = False
    if len(uzlist)>0 and len(vzlist)>0:
        uz0 = uzlist[0]; vz0 = vzlist[len(vzlist)-1]
        if (uz0-vz0)!=0 and 0<vz0<800 and 2000<uz0<3000:
            isassigned=True
            return 'VeloUT' #best because most precise slope (long)
    if isassigned == False:
        return -1
    
def isutut(uzlist,vzlist):
    isassigned = False
    if len(uzlist)>1:
        uz0 = uzlist[0]; uz1 = uzlist[len(uzlist)-1]
        if (uz1-uz0)!= 0 and 2000<uz0<uz1<3000:
            isassigned=True
            return 'UTUT' #better than velovelo because hits are closer -> error propagated less
    if isassigned == False:
        return -1

    
def isvelovelo(uzlist,vzlist):
    isassigned = False
    if len(vzlist)>1:
        vz0 = vzlist[0]; vz1 = vzlist[len(vzlist)-1]
        if (vz1-vz0)!=0 and 0<vz0<vz1<800:
            isassigned=True
            return 'VeloVelo'
    if isassigned == False:
        return -1

In [ ]:
#function that checks if a point (format: (x,y)) is inside the MightyIT region

def MightyITregion():
    dx = 540
    dy = 200
    
    polygon = Polygon([(-4*dx,1*dy),(-3*dx,1*dy),(-3*dx,2*dy),(-2*dx,2*dy),(-2*dx,3*dy),(-1*dx,3*dy),(-1*dx,4*dy),
                      (1*dx,4*dy),(1*dx,3*dy),(2*dx,3*dy),(2*dx,2*dy),(3*dx,2*dy),(3*dx,1*dy),(4*dx,1*dy),
                      (4*dx,-1*dy),(3*dx,-1*dy),(3*dx,-2*dy),(2*dx,-2*dy),(2*dx,-3*dy),(1*dx,-3*dy),(1*dx,-4*dy),
                      (-1*dx,-4*dy),(-1*dx,-3*dy),(-2*dx,-3*dy),(-2*dx,-2*dy),(-3*dx,-2*dy),(-3*dx,-1*dy),(-4*dx,-1*dy)])
    return polygon
    
def is_inregion(point):
    point = Point(point[0], point[1])
           
    region = MightyITregion()
    if region.contains(point):
        return True
    else:
        return False
    

In [ ]:
def Slopexy(x0, y0, z0, x1, y1, z1):
    slopex=(x1 - x0)/(z1 - z0)
    slopey=(y1 - y0)/(z1 - z0)
    return slopex,slopey

In [ ]:
#function that finds intersection of velo and t track for a longtrack, on the bending plane
#outputs the z coordinate (zmag)

def findZmag(xv0, yv0, zv0, xv1, yv1, zv1, xt0, yt0, zt0, xt1, yt1, zt1): #to ask: velo p?
    tyV=(yv1 - yv0)/(zv1 - zv0)
    txV=(xv1 - xv0)/(zv1 - zv0)
    tyT=(yt1 - yt0)/(zt1 - zt0)
    txT=(xt1 - xt0)/(zt1 - zt0)
    
    #extrapolate velo track from v1 to t0 with velo slope
    xV_zt0 = xv1 + txV*(zt0-zv1)
    
    #extrapolate t track from t0 to v1 with t track slope
    xT_zv1 = xt0 + txT*(zv1-zt0)
    
    velo_extr = LineString([(xv1,zv1), (xV_zt0,zt0)])
    t_extr = LineString([(xT_zv1,zv1), (xt0,zt0)])
    
    #todo: deal with no intersection situation
    if velo_extr.intersection(t_extr):
        zmag = velo_extr.intersection(t_extr).y #it is actually z
    else:
        zmag = -1
   
    return zmag

In [ ]:
#function which calculates the chi2

def ChiSquareVeloT(xv0, yv0, zv0, xv1, yv1, zv1, xt0, yt0, zt0, xt1, yt1, zt1 , zmag, sigmax, sigmay, sigmatx, sigmaty, check_if_inwindow=False, smearing=False): #to ask: velo p?
    
    #gaussian smearing - pixel resolutions x and y according to if in mightyit region or outside
    if smearing == True:
        if (is_inregion([xt0,yt0]) == True):
            xt0 = gRandom.Gaus(xt0,resx_mit)
            yt0 = gRandom.Gaus(yt0,resy_mit)
        else:
            xt0 = gRandom.Gaus(xt0,resx_sf)
            yt0 = gRandom.Gaus(yt0,resy_sf)
            
        if (is_inregion([xt1,yt1]) == True):
            xt1 = gRandom.Gaus(xt1,resx_mit)
            yt1 = gRandom.Gaus(yt1,resy_mit)
        else:
            xt1 = gRandom.Gaus(xt1,resx_sf)
            yt1 = gRandom.Gaus(yt1,resy_sf)
            
    tyV=(yv1 - yv0)/(zv1 - zv0)
    txV=(xv1 - xv0)/(zv1 - zv0)
    tyT=(yt1 - yt0)/(zt1 - zt0)
    txT=(xt1 - xt0)/(zt1 - zt0)

    #zmag calculations
    xVzmag=xv1+(zmag-zv1)*txV
    yVzmag=yv1+(zmag-zv1)*tyV
       
    
    #chi2 calculations
    txPre=(xt0-xVzmag)/(zt0-zmag)
    tyPre=(yt0-yVzmag)/(zt0-zmag)
    
    #extrapolated point from zmag to t station, with the same slope of the t track
    xT_ex=xVzmag+(zt0-zmag)*txT
    #yT_ex=yVzmag+(zt0-zmag)*tyT
    #from velo to t track with velo slope
    yT_ex=yv1+(zt0-zv1)*tyV
    
    #square sigmas
    sigma2x=pow(sigmax,2)
    sigma2y=pow(sigmay,2)
    sigma2tx=pow(sigmatx,2)
    sigma2ty=pow(sigmaty,2)

    #ŧesting: complete chi2 (slopes and coordinates)
    chi2 = (txPre-txT)*(txPre-txT)/sigma2tx+(tyPre-tyT)*(tyPre-tyT)/sigma2ty + (yT_ex-yt0)*(yT_ex-yt0)/sigma2y + (xT_ex-xt0)*(xT_ex-xt0)/sigma2x 

    #check if in window
    if (check_if_inwindow == True):
        
        rangex = sigmax*2
        rangey = sigmay*2
        rangetx = sigmatx*2
        rangety = sigmaty*2
        
        #if in window: return chi2, else placeholder -1
        if(abs(xT_ex-xt0)<rangex and abs(yT_ex-yt0)<rangey and (txPre-txT)<rangex and abs(tyPre-tyT)<rangey):
            return chi2, (xT_ex-xt0), (yT_ex-yt0), (txPre-txT), (tyPre-tyT)
        else:
            return (-1, -1, -1, -1, -1)
            
    #no need to check 
    else:
        return chi2, (xT_ex-xt0), (yT_ex-yt0), (txPre-txT), (tyPre-tyT) 

In [ ]:
# pixel smearing in mightyIT
resx_mit = 0.1/TMath.Sqrt(12)
resy_mit = 0.4/TMath.Sqrt(12)
# strips smearing outside mightyIT
resx_sf = 0.25/TMath.Sqrt(12)
resy_sf = 0.25/TMath.Cos(5.*np.pi/180.) #takes into account 5° tilting